In [ ]:
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.drivers import PySCFDriver
from typing import List
from qiskit import Aer
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD,UCC
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit.circuit import QuantumCircuit
from qiskit.primitives import Estimator  
import numpy as np
import logging
import sys,pickle,copy,re
from qiskit_nature import settings
settings.use_pauli_sum_op = False
sys.path.append('..')
from qiskit.quantum_info import SparsePauliOp
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
import numpy as np
from copy import  deepcopy

class BaseFermonicAdaptVQEqqq():
    def __init__(self, ES_problem: ElectronicStructureProblem,threshold:float=1e-6,max_iter:int=40) -> None:
        self.mapper = JordanWignerMapper()
        self.es_problem = ES_problem
        self.estimator = Estimator()
        self.threshold = threshold
        self.max_iteration = max_iter
        self.hamiltonian = self.mapper.map(ES_problem.hamiltonian.second_q_op())
        self.init_state_hf = HartreeFock(num_particles=self.es_problem.num_particles,
                                         num_spatial_orbitals=self.es_problem.num_spatial_orbitals,
                                         qubit_mapper=self.mapper)
        self.n_qubit = self.init_state_hf.num_qubits
        self.adapt_ansatz=QuantumCircuit(self.n_qubit)
        self.adapt_ansatz.append(self.init_state_hf,range(self.n_qubit))#存放动态增长的ansatz circuit的instruction
        self.converageflag = False #是否收敛的标志
        self._already_pick_index = [] #目前已经挑选的index列表
        self.finnal_pool_op=[] 
        self.circuits = []
        self.solver = VQE(estimator=Estimator(),
                          ansatz=self.init_state_hf,
                          optimizer=SLSQP())
        self.pool_init()
        self.first_step()


    def pool_init(self):
        uccsd = UCC(num_particles=self.es_problem.num_particles,
                    num_spatial_orbitals=self.es_problem.num_spatial_orbitals,
                    excitations='sd',qubit_mapper=JordanWignerMapper(),initial_state=self.init_state_hf)
        
        self.fermonic_pool = [EvolvedOperatorAnsatz(operators=JordanWignerMapper().map(i),name='Fermonic_op'+'{:02d}'.format(index),\
            parameter_prefix='{:02d}'.format(index)) for index,i in enumerate(uccsd.excitation_ops())]
        self.finnal_pool_op = [self.mapper.map(i) for i in uccsd.excitation_ops()]
        print(f'fermonicpool 创建完毕,size={len(self.finnal_pool_op)}个')
        self.commutors= [1j * (self.hamiltonian @ exc - exc @ self.hamiltonian) for exc in self.finnal_pool_op]


    @staticmethod
    # 参数是 每一轮求得的梯度的最大值
    #判断是否达到收敛条件
    def check_gradient_converge(value:List,criterion: float = 1e-3) -> bool:
        converge = value.max()
        if converge > criterion:
            print(f'没有达到收敛标准,标准为{criterion},当前值为{converge}')
            return False
        else:
            print(f'达到收敛标准,标准为{criterion},当前值为{converge}')
            return True


    def first_step(self):
        print('现在挑选第一个算符...')
        self.circuits = []
        n = self.n_qubit
        for i in range(len(self.finnal_pool_op)): 
            qc = QuantumCircuit(n)
            qc.append(self.init_state_hf, range(n))
            self.circuits.append(qc)

        job = self.estimator.run(circuits=self.circuits,observables=self.commutors)
        result = job.result()
        value =np.abs(result.values)
        k = np.argmax(value)
        print(f'初始化结果:第{np.argmax(value)}项被选定,此项梯度最大,为{value[k]}')
        self._already_pick_index.append(k)
        self.iteration_index = int(1)
        self.next_operator= EvolvedOperatorAnsatz(operators=self.finnal_pool_op[k],parameter_prefix='{:02d}'.format(0),name="Fermonic"+'_'+str(k))
        #此时更新adapt_ansatz
        self.adapt_ansatz.append(self.next_operator,range(self.n_qubit))
        self.solver.ansatz = self.adapt_ansatz
        self.solver.initial_point=[0.0]
        self.vqe_result = self.solver.compute_minimum_eigenvalue(self.hamiltonian)
        self.optimal_parameter = self.vqe_result.optimal_point.tolist()
        print(f'第一轮的优化结果:optimal_point={self.vqe_result.optimal_point}')
        print(self.adapt_ansatz)
        print(self.circuits[k])
        # for i in range(len(self.finnal_pool_op)): 
        #     self.circuits[i] = self.circuits[i].append(self.adapt_ansatz.bind_parameters(self.optimal_parameter))
    
    
    def next_operator1(self, bound_circuit: QuantumCircuit):
        print(f'No.{self.iteration_index}轮,正在准备挑选下一块算符...')
        job = self.estimator.run(circuits=[bound_circuit]*len(self.commutors), observables=self.commutors)
        result = job.result()
        value = np.abs(result.values)
        k = np.argmax(value)
        #判断即将选择的算符是否已经陷入循环
        if self.check_gradient_converge(value=value,criterion=self.threshold):
            print(f'梯度满足或者检测到循环!')
            self.converageflag=True
            print(f'已经达到收敛条件!')
            return
        
        else:
            print(f'第{self.iteration_index}轮中梯度最大项为第{k}项,已被选入算符池...')  
            self.optimal_parameter.append(0.0)      
            self.adapt_ansatz.append(EvolvedOperatorAnsatz(operators=self.finnal_pool_op[k],
                                                           parameter_prefix='{:02d}'.format(self.iteration_index),name='Fermonic_'+str(k)),range(self.n_qubit))
            self._already_pick_index.append(k)
    

    def run(self):
        #vqe_result=[]
        while(self.converageflag==False and self.iteration_index<self.max_iteration):
            print(f'------------第{self.iteration_index}轮--------------')
            print(f'已经选好的index是{self._already_pick_index}')

            self.solver.ansatz = self.adapt_ansatz
            self.solver.initial_point=self.optimal_parameter            
            self.next_operator1(bound_circuit=self.solver.ansatz.bind_parameters(self.optimal_parameter))
            self.vqe_result = self.solver.compute_minimum_eigenvalue(self.hamiltonian)
            self.iteration_index += 1
            print(self.adapt_ansatz)
            print(self.vqe_result.optimal_point)
        self.converageflag=True
        print(f'✔Adapt VQE算法结果={self.vqe_result.optimal_value}')

        

                
            
            
            
    

In [ ]:
dist = 1.5
molecule = MoleculeInfo(
    ["H", "H","H","H"], [(0.0, 0.0, 0.0), (0.0, 0.0, dist), (0.0, 0.0,dist*2), (0.0, 0.0, dist*3)],
    # ["H", "H"], [(0.0, 0.0, 0.0), (0.0, 0.0, dist),],
    #['Be','H','H'],[(0.0,0.0,0.0),(0.0,0.0,dist),(0.0,0.0,-dist)],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)
driver = PySCFDriver().from_molecule(molecule)
problem = driver.run()
mapper = JordanWignerMapper() 


In [ ]:
f1 = BaseFermonicAdaptVQEqqq(ES_problem=problem,max_iter=30,threshold=1e-3)

In [ ]:
f1.run()